# Exploration des Variables Catégorielles



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration esthétique
sns.set_theme(style="whitegrid")
pd.set_option('display.max_rows', 100)

# Chemins (Adaptés à la structure du projet)
DATA_DIR = Path("../data")
ARTIFACTS_DIR = Path("../artifacts/baseline_v0")

## 1. Chargement des Données
Nous utilisons le fichier `train_cleaned.parquet` généré par le pipeline, car il contient les données déjà nettoyées (minuscules, espaces retirés, etc.).

In [2]:
try:
    df = pd.read_parquet(ARTIFACTS_DIR / "train_cleaned.parquet")
    print(f"Données chargées : {df.shape[0]} lignes, {df.shape[1]} colonnes")
except FileNotFoundError:
    print("Fichier cleaned introuvable, chargement du CSV brut...")
    df = pd.read_csv(DATA_DIR / "Train.csv")
    # Petit nettoyage à la volée si on charge le brut
    cat_cols = ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code']
    for c in cat_cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.casefold()

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [5]:

df = pd.read_csv(DATA_DIR / "Train.csv")
# Petit nettoyage à la volée si on charge le brut
cat_cols = ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code']
for c in cat_cols:
    if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.casefold()

## 2. Analyse par Variable
Nous allons itérer sur les colonnes d'intérêt pour afficher :
1. Le nombre de catégories uniques.
2. Le Top 20 des valeurs les plus fréquentes.
3. Un graphique de distribution.

In [7]:
COLS_TO_INSPECT = ['marital_status', 'branch_code', 'occupation_category_code', 'occupation_code']

for col in COLS_TO_INSPECT:
    if col not in df.columns:
        continue
        
    print(f"\n{'='*40}\nVariable : {col}\n{'='*40}")
    
    # Compte des valeurs
    counts = df[col].value_counts(dropna=False)
    n_unique = df[col].nunique()
    for v, c in counts.items():
        print(f"{v}: {c}")
    print(f"Nombre de valeurs uniques : {n_unique}")


Variable : marital_status
m: 22913
s: 3267
u: 2474
w: 301
d: 135
r: 34
p: 7
f: 1
Nombre de valeurs uniques : 8

Variable : branch_code
748l: 8136
94kc: 3658
uaod: 3507
e5sw: 3185
30h5: 3151
xx25: 2805
1x1h: 1303
o67j: 1125
zfer: 747
49bm: 621
9f9t: 429
boas: 347
x23b: 99
o4jc: 10
eu3l: 9
Nombre de valeurs uniques : 15

Variable : occupation_category_code
t4ms: 18619
90qi: 5552
56si: 3655
l44t: 628
jd7x: 576
ahh5: 102
Nombre de valeurs uniques : 6

Variable : occupation_code
2a7i: 7089
0kid: 3969
sst3: 2711
uj5t: 2079
0ojm: 1565
bpsa: 1091
qzyx: 856
31jw: 683
0b60: 566
bp09: 525
ruft: 456
6pe7: 448
6e4h: 438
u37o: 402
5fpk: 347
3nhz: 332
j9sy: 323
n2zz: 261
ymgt: 248
opvx: 240
wmtk: 235
b8w8: 225
pmai: 217
4w0d: 162
7udq: 152
fsxg: 124
p4md: 124
apo0: 117
i31i: 116
k5lb: 112
1nfk: 111
f57o: 102
73ac: 100
xc1n: 99
s9ku: 96
1msv: 88
q57t: 72
vreh: 72
u9rx: 69
t6ab: 65
nqw1: 61
9hd1: 60
iqfs: 59
e39i: 57
ss6d: 56
7g9m: 55
we7u: 50
c8f6: 47
ji64: 40
js7m: 40
wv7u: 39
2be6: 39
dprv: 34
sf1x

In [ ]:
a

In [ ]:
COLS_TO_INSPECT = ['marital_status', 'branch_code', 'occupation_category_code', 'occupation_code']

for col in COLS_TO_INSPECT:
    if col not in df.columns:
        continue
        
    print(f"\n{'='*40}\nVariable : {col}\n{'='*40}")
    
    # Compte des valeurs
    counts = df[col].value_counts(dropna=False)
    n_unique = df[col].nunique()
    
    print(f"Nombre de valeurs uniques : {n_unique}")
    print("\nTop 10 valeurs les plus fréquentes :")
    print(counts.head(10))
    
    # Visualisation (Top 20 pour lisibilité)
    plt.figure(figsize=(10, 5))
    sns.barplot(x=counts.head(20).index, y=counts.head(20).values, palette="viridis")
    plt.title(f"Distribution de {col} (Top 20)")
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("Nombre de clients")
    plt.show()

## 3. Focus : Mapping des Métiers
On regarde le lien entre `occupation_code` (très granulaire) et `occupation_category_code` (plus général) pour essayer de deviner le sens des codes.

In [ ]:
# On regarde pour chaque catégorie, quels sont les codes métiers les plus fréquents
if 'occupation_category_code' in df.columns and 'occupation_code' in df.columns:
    for cat in df['occupation_category_code'].unique():
        sub_df = df[df['occupation_category_code'] == cat]
        top_jobs = sub_df['occupation_code'].value_counts().head(5).index.tolist()
        print(f"Catégorie '{cat}' contient principalement les codes : {top_jobs}")